<a href="https://colab.research.google.com/github/atharvajpatel/AI-Hackathon-Ignition/blob/master/Scikit_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning vs Regular Software:

Machine Learning takes input and output and finds equation (data is output, that is why it is so important in ML)
Regular software takes input and equation and finds output

Example:
- Machine Learning can derive the equation of celsius to fahrenheit or vice versa and then can eventually accurately predict the answer by testing the model and putting a lot of data
- Regular software will take the equation and either celsius or fahrenheit and tell you the other one.

Deep Learning vs Machine Learning:
Machine learning uses algorithms to parse data, learn from that data, and make informed decisions based on what it has learned. Deep learning structures algorithms in layers to create an "artificial neural network” that can learn and make intelligent decisions on its own.

Machine Learning Steps:

1) What question are we trying to answer

2) Find data to help answer that question

3) Process data

4) Build Model

5) Test/Evauluate model

6) Improve model further

Note:
For the competition they are actually going to supply us with the data so we don't have to worry about finding it (i'm pretty sure)

###Natural Language Processing model (predict whether a review is positive or negative)

In [110]:
#import statements for the libraries and initalizing the file into df
#Don't know how to work with json, so converted the data into csv. Data is in comment section of Keith's video
import json
import pandas as pd
import io
import random
from google.colab import files
#Scikit imports
#Don't import entire library because its too large and will take too long to load
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import svm
from sklearn.metrics import f1_score

In [100]:
#All the important classes to manipulate data


#Enum class so you can access each value in reviews
class Sentiment:
  NEG = "Negative"
  NEU = "Neutral"
  POS = "Positive"

#Review class to get the sentiment
class Review:
  text = " "
  score = 0

  def __init__(self, text, score):
    self.score = score
    self.text = text
    self.sentiment = self.getSentiment() 

  def getSentiment(self):
    if self.score == 1 or self.score == 2:
      return Sentiment.NEG
    elif self.score == 3:
     return Sentiment.NEU
    else:
      return Sentiment.POS

#Review class to even out all the data
class ReviewContainer:
  def __init__(self, reviews):
    self.reviews = reviews
  
  def getText(self):
    return [x.text for x in self.reviews]
  
  def getSentiment1(self):
    return [x.sentiment for x in self.reviews]

  def evenly_distribute(self):
    negative = list(filter(lambda x: x.sentiment == Sentiment.NEG, self.reviews))
    positive = list(filter(lambda x: x.sentiment == Sentiment.POS, self.reviews))
    positive_shrunk = positive[:len(negative)]
    self.reviews = negative + positive_shrunk
    random.shuffle(self.reviews)


In [ ]:
#Loading Data into from file to dataframe

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['Books_small_10000.csv'])) #Puts all the data in one variable

#Parsing the data for all the important stuff. 2D list with the review, rating, and sentiment
reviews = []
for index, rows in df.iterrows():
  reviews.append(Review(df.iloc[index, 5],df.iloc[index,6]))

#Just to make sure that the file loaded in properly
print(reviews[0].text)



In [113]:
# Entire function is basically the actual algorithm that does the model

#Add this at the end
def main():
  
  #Model needs training and testing data so we pass in both

  train, test = train_test_split(reviews, test_size = 0.33, random_state = 42)

  trainC = ReviewContainer(train)
  trainC.evenly_distribute()
  testC = ReviewContainer(test)
  testC.evenly_distribute()

  #X is what we pass in, Y is what our results are. Pass a review get a sentiment
  trainX = trainC.getText()
  trainY = trainC.getSentiment1()

  #Same thing for testing data
  testX = testC.getText()
  testY = testC.getSentiment1()


  #Using vectorizer to sort binary digits

  #vectorizer = CountVectorizer()
  vectorizer = TfidfVectorizer()
  trainX_vector = vectorizer.fit_transform(trainX)
  testX_vector = vectorizer.transform(testX)


  #Classifying the data

  #SVM = support vector machine
  clf_svm = svm.SVC(kernel='linear')
  clf_svm.fit(trainX_vector, trainY)

main()


In [ ]:
#Analyzing the percent of data that is correct and f1
print(clf_svm.predict(testX_vector[0]))

print(clf_svm.score(testX_vector, testY))

#f-1 score

print(f1_score(testY, clf_svm.predict(testX_vector), average = None))

In [ ]:
#Final product

def final(list1):
  new_test = vectorizer.transform(list1)
  return clf_svm.predict(new_test)

user_val = input("Enter some feedback(press enter to exit): ")
list1 = []
list1.append(user_val)
print(final(list1))